In [ ]:
#@title Downloading helper files...<p>(Run this cell first!)
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/priv-sec/twai/master/exercise7_data/binary_model.py", "binary_model.py")
urllib.request.urlretrieve("https://raw.githubusercontent.com/priv-sec/twai/master/exercise7_data/cifar10_model.py", "cifar10_model.py")
urllib.request.urlretrieve("https://raw.githubusercontent.com/priv-sec/twai/master/exercise7_data/MIA.py", "MIA.py")

In [ ]:
# JUST ONCE
!pip install adversarial-robustness-toolbox

## Load Cifar10 model

In [ ]:
import torch
from torchvision import datasets, transforms

import numpy as np

from cifar10_model import Cifar10Net, train_cifar10_model

from MIA import get_mia_accuracy

from art.estimators.classification.pytorch import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod



cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")

train_kwargs = {
    'batch_size': 32,
    'shuffle': True
    }

test_kwargs = {
    'batch_size': 1000,
    'shuffle': True
    }

if cuda_available:
    cuda_kwargs = {
        'num_workers': 2,
        'pin_memory': True,
        'shuffle': True
        }
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset))
data = next(iter(train_loader))
train_dataset, train_targets = data[0].numpy(), data[1].numpy()
train_dataset.reshape(-1, 3, 32, 32)


def to_loader(x, y, kwargs):
  dataset = torch.utils.data.TensorDataset(torch.Tensor(x).type(torch.FloatTensor),
                                                 torch.Tensor(y).type(torch.LongTensor) )
  return torch.utils.data.DataLoader(dataset, **kwargs)


def create_dataset(size: int):

  # 1 < size < 25000

  x_train, x_test = train_dataset[:size], train_dataset[size:size*2]
  y_train, y_test = train_targets[:size], train_targets[size:size*2]

  train_loader = to_loader(x_train, y_train, train_kwargs)
  test_loader = to_loader(x_test, y_test, test_kwargs)

  x_train_dup = np.concatenate([x_train, x_train])
  y_train_dup = np.concatenate([y_train, y_train])

  x_test_dup = np.concatenate([x_test, x_test])
  y_test_dup = np.concatenate([y_test, y_test])

  train_loader_dup = to_loader(x_train_dup, y_train_dup, train_kwargs)
  test_loader_dup = to_loader(x_test_dup, y_test_dup, test_kwargs)

  return (x_train, y_train), (x_test, y_test), (train_loader, test_loader),\
         (x_train_dup, y_train_dup), (x_test_dup, y_test_dup), (train_loader_dup, test_loader_dup)


(x_train, y_train), (x_test, y_test), (train_loader, test_loader), \
(x_train_dup, y_train_dup), (x_test_dup, y_test_dup), (train_loader_dup, test_loader_dup) = create_dataset(10000)

# Exercise

In [ ]:
def to_art_model(model):
  """
  Converts the PyTorch model into an ART model.
  """

  model = torch.nn.DataParallel(model)
  
  # Loss function of our Cifar10 model
  criterion = # TODO #

  # Optimizer of our Cifar10 model
  optimizer = # TODO #

  # Min and max pixel value of the Cifar10 pictures after the transformation
  clip_values = # TODO Hint: A tuple # 

  # The shape of a single picture after reshaping
  input_shape = # TODO #

  # Amount of classes
  classes = # TODO #

  art_model = # TODO Hint: Init PyTorchClassifier #
  
  return art_model

In [ ]:
# Train without duplicates #

# Train Cifar10 model
model = Cifar10Net().to(device)
# TODO: Train Cifar19 model for 15 epochs #

# Get accuracy of the MIA
# TODO: Use <get_mia_accuracy> Hint: size=10000 #

# Get AVG epsilon used to generate adversarial examples via FGSM
x, y = np.concatenate([x_train, x_test], np.concatenate([y_train, y_test])
art_model = to_art_model(model)
fgsm = # TODO: Init FastGradientMethod with eps=1 and eps_step=0.01 #
adv_examples = # TODO: Generate ADV_Examples on x and y #

# Compute the avg epsilon used to generate all adversarial examples
avg_eps = # TODO: Compute AVG epsilon used to turn pictures into adversarial examples #

print("### FGSM RESULTS ###\n")
print("AVG EPS: ", end="")
print(avg_eps)

In [ ]:
# Train with duplicates #

# Train Cifar10 model
model = Cifar10Net().to(device)
# TODO: Train Cifar19 model for 15 epochs #

# Get accuracy of the MIA
# TODO: Use <get_mia_accuracy> Hint: size=20000 #

# Get AVG epsilon used to generate adversarial examples via FGSM
x, y = np.concatenate([x_train_dup, x_test_dup]), np.concatenate([y_train_dup, y_test_dup])
art_model = to_art_model(model)
fgsm = # TODO: Init FastGradientMethod with eps=1 and eps_step=0.01 #
adv_examples = # TODO: Generate ADV_Examples on x and y #

# Compute the avg epsilon used to generate all adversarial examples
avg_eps = # TODO: Compute AVG epsilon used to turn pictures into adversarial examples #

print("### FGSM RESULTS ###\n")
print("AVG EPS: ", end="")
print(avg_eps)